# SQL II: Grouping and Joining

_August 10, 2020_

Agenda:
- Practice with Grouping in SQLite
- Practice with different types of Joins in SQLite

<img src = "https://media.giphy.com/media/3oKIPnAiaMCws8nOsE/giphy.gif">

In [1]:
import sqlite3
import pandas as pd
import seaborn as sns
import numpy as np

## Part I. Grouping
Just like pandas, Sql support different types of grouping statements for performing aggregate functions and allow us to calculate various statistics for data.

Syntax:
`SELECT * FROM tables GROUP BY column`

In [2]:
# instantiate some data
auto = pd.read_csv('auto-mpg.csv')
tips = sns.load_dataset('tips')
small_quiz = pd.DataFrame({"students":["Ben","Toni","Anton"],
                          "projects":[1,2,1],
                          "grades":np.random.randint(80,100,3)})
small_grades = pd.DataFrame({"students":["Toni","Jamil","Jeffrey"],
                            "quiz_score":np.random.randint(0,10,3)})

In [3]:
conn = sqlite3.connect('aggregate.db')
c = conn.cursor()

In [40]:
# insert these datasets into our db
auto.to_sql('auto', con=conn)
tips.to_sql('tips', con=conn)
small_grades.to_sql('small_grade', con=conn)
small_quiz.to_sql('small_quiz',con=conn)

/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


ValueError: Table 'auto' already exists.

In [63]:
# for the tips df/table, get the total number of man and woman
tips.groupby('sex').sex.value_counts()
#tips.sex.value_counts()
#tips.groupby(['sex']).size()


sex     sex   
Male    Male      157
Female  Female     87
Name: sex, dtype: int64

In [64]:
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


Syntax: `SELECT ___ FROM ___ GROUP BY ___` 

In [65]:
# in sql 
query = """SELECT sex,COUNT(sex)From tips GROUP BY sex"""
c.execute(query).fetchall()

[('Female', 87), ('Male', 157)]

In [66]:
# using pandas, select the average amount tipped for time
tips.groupby('time').tip.mean()

time
Lunch     2.728088
Dinner    3.102670
Name: tip, dtype: float64

In [67]:
# use sql to recreate this query
query = """SELECT time, AVG(tip)FROM tips GROUP BY time"""
c.execute(query).fetchall()

[('Dinner', 3.102670454545454), ('Lunch', 2.7280882352941176)]

We can also use `GROUP BY` with certain conditions, in conjunction with filtering and ordering.

You can use :

`SELECT ___ FROM ___ WHERE ___ GROUP BY ___ `

`SELECT ___ FROM ___ GROUP BY ___ HAVING ____ `

In [68]:
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [69]:
# use pandas, get the average total bills for a party larger than 2 for every size of party
tips.groupby(['size']).total_bill.mean().sort_index()[2:]
# using pandas

size
3    23.277632
4    28.613514
5    30.068000
6    34.830000
Name: total_bill, dtype: float64

In [70]:
# use sql to recreate this query
query =  """SELECT time, AVG(total_bill)FROM tips WHERE size >2 GROUP BY size"""
#query =  """SELECT time, AVG(total_bill)FROM tips GROUP BY size HAVING size>2"""
c.execute(query).fetchall()

[('Dinner', 23.27763157894737),
 ('Dinner', 28.61351351351351),
 ('Lunch', 30.068),
 ('Lunch', 34.83)]

In [71]:
# your turn - use pandas, get the maximum value of total bill for female non smoker
tips.groupby(['sex','smoker']).total_bill.max()['Female']['No']

35.83

In [72]:
# how to get this without using group by?
tips[(tips.sex== "Female")& (tips.smoker == 'No')]['total_bill'].max()

35.83

In [73]:
# use sql to recreate this query
query = """SELECT sex, max(total_bill) From tips Where sex ='Female' AND smoker = 'No'"""
c.execute(query).fetchall()

[('Female', 35.83)]

In [74]:
# your turn - use pandas, get the average tip values for different size groups, and show top 2
# most amount of average tips
tips.groupby(['size']).tip.mean().sort_values(ascending = False)


size
6    5.225000
4    4.135405
5    4.028000
3    3.393158
2    2.582308
1    1.437500
Name: tip, dtype: float64

In [75]:
# use sql 
query = """SELECT size, AVG(tip)FROM tips GROUP BY size ORDER BY AVG(tip) DESC LIMIT 2""" 
c.execute(query).fetchall()

[(6, 5.225), (4, 4.135405405405407)]

## Part II. Join

Based on this diagram, how would you explain the different types of joins?

<img src = 'sql-joins.png' width = 400>

In [76]:
print(small_grades)
print(small_quiz)

  students  quiz_score
0     Toni           6
1    Jamil           8
2  Jeffrey           2
  students  projects  grades
0      Ben         1      80
1     Toni         2      98
2    Anton         1      85


In [77]:
# left join - join small quiz with small grades, on student name
small_grades.merge(small_quiz, on = 'students', how = "left")

# if i run this command, what would happen? how many observations/record?

,students,quiz_score,projects,grades
0,Toni,6,2.0,98.0
1,Jamil,8,NaN,NaN
2,Jeffrey,2,NaN,NaN


In [78]:
# sql 
query = """SELECT * FROM small_grade LEFT JOIN small_quiz ON small_grade.students == small_quiz.students"""
c.execute(query).fetchall()

[(0, 'Toni', 6, 1, 'Toni', 2, 98),
 (1, 'Jamil', 8, None, None, None, None),
 (2, 'Jeffrey', 2, None, None, None, None)]

In [79]:
# inner join and get the student's grades
small_grades.merge(small_quiz, on = 'students', how = 'inner')

,students,quiz_score,projects,grades
0,Toni,6,2,98


In [82]:
query = """SELECT * FROM small_quiz JOIN small_grade ON small_grade.students = small_quiz.students"""
c.execute(query).fetchall()

[(1, 'Toni', 2, 98, 0, 'Toni', 6)]

In [81]:
# your turn - use right join and get all entries

# pandas

In [ ]:
# sql 

### Level up 
1. Subqueries (including anti-joins)
2. Get median (using OFFSET)

In [ ]:
# anti-join :get all the entries that are in the left table but not in the right table 
query = """SELECT left_table.id FROM `left table` WHERE left_table.id not in (SELECT id FROM right_table) """

In [ ]:
# get median - use offset 
query = """SELECT col FROM table ORDER BY col LIMIT 1 OFFSET (SELECT COUNT(*) FROM MyTable) / 2"""